LiH molecule ground state with Active Space Transformation.

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

# 1. Define molecule LiH
driver = PySCFDriver(atom="Li 0 0 0; H 0 0 1.6", basis="sto-3g")  # 1.6 Å bond
es_problem = driver.run()

# 2. Apply Frozen Core Approximation (active space transformer)
# For LiH, we freeze Li 1s core orbital and keep 2 electrons in 2 spatial orbitals
transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=2)
es_problem_reduced = transformer.transform(es_problem)

# 3. Mapper
mapper = JordanWignerMapper()

# 4. Ansatz (UCCSD + HartreeFock)
ansatz = UCCSD(
    es_problem_reduced.num_spatial_orbitals,
    es_problem_reduced.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem_reduced.num_spatial_orbitals,
        es_problem_reduced.num_particles,
        mapper,
    ),
)
# 5. Optimizer
optimizer = SLSQP(maxiter=200)

# 6. VQE solver
estimator = Estimator()
vqe = VQE(estimator, ansatz, optimizer)

solver = GroundStateEigensolver(mapper, vqe)

# 7. Solve ground state
res = solver.solve(es_problem_reduced)

print(" LiH Ground State Energy (Frozen Core)")
print(res)
print("Ground state energy (Hartree):", res.total_energies[0].real)
